##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Universal Sentence Encoder


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/s?q=google%2Funiversal-sentence-encoder%2F4%20OR%20google%2Funiversal-sentence-encoder-large%2F5"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub models</a>
  </td>
</table>

This notebook illustrates how to access the Universal Sentence Encoder and use it for sentence similarity and sentence classification tasks.

The Universal Sentence Encoder makes getting sentence level embeddings as easy as it has historically been to lookup the embeddings for individual words. The sentence embeddings can then be trivially used to compute sentence level meaning similarity as well as to enable better performance on downstream classification tasks using less supervised training data.


## Setup

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [ ]:
%%capture
!pip3 install seaborn

# Try T5

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

input_sent = [' sentiment review: I love chocolate,hi,hi', ' sentiment review: I love chocolate']
labels = ['positive ', 'negative ']
input_ids = tokenizer(input_sent, return_tensors='pt',padding=True).input_ids
target_ids = tokenizer(labels, return_tensors='pt').input_ids
outputs = model(input_ids=input_ids, labels=target_ids, use_cache=False, return_dict=True)

In [ ]:
!git config --global credential.helper store

In [ ]:
import json
import numpy as np
messages = [
    "3 agents occupy 3 landmarks",
    "4 agents occupy 4 landmarks",
    "5 agents occupy 5 landmarks",
    "3 agents form a regular polygon near the landmark",
    "4 agents form a regular polygon near the landmark",
    "5 agents form a regular polygon near the landmark",
    "3 agents compete with 3 adversaries to occupy landmarks",
    "4 agents compete with 4 adversaries to occupy landmarks",
    "5 agents compete with 5 adversaries to occupy landmarks",
    "4 agents chase 2 adversaries",
    "2 agents avoid 4 adversaries",
    "6 agents chase 3 adversaries",
    "3 agents avoid 6 adversaries",
    # "5 agents push 5 enemies away from 5 landmarks",
    "Agents employ various strategies and tactics to effectively deter adversaries from approaching or attacking significant landmarks.",
]
codes = [
    "011100001000",
    "100100001000",
    "101100001000",
    "011010000100",
    "100010000100",
    "101010000100",
    "011001001000",
    "100001001000",
    "101001001000",
    "100000101000",
    "010000011000",
    "110000101000",
    "011000011000",
    "101111111000",
]
labels=["hi" for _ in range(len(messages))]
input_ids = tokenizer(messages, return_tensors='pt',padding=True).input_ids
target_ids = tokenizer(labels, return_tensors='pt').input_ids
outputs = model(input_ids=input_ids, labels=target_ids, use_cache=False, return_dict=True)
message_embeddings=outputs['encoder_last_hidden_state'].mean(dim=1).cpu().detach()
data = {}
for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print(message_embedding)
    data[codes[i]] = message_embedding
    # data[messages[i]] = message_embedding

with open('T5.json', 'w') as file:
  json.dump(data, file)


# Try random

In [ ]:
import json
import numpy as np
messages = [
    "3 agents occupy 3 landmarks",
    "4 agents occupy 4 landmarks",
    "5 agents occupy 5 landmarks",
    "3 agents form a regular polygon near the landmark",
    "4 agents form a regular polygon near the landmark",
    "5 agents form a regular polygon near the landmark",
    "3 agents compete with 3 adversaries to occupy landmarks",
    "4 agents compete with 4 adversaries to occupy landmarks",
    "5 agents compete with 5 adversaries to occupy landmarks",
    "4 agents chase 2 adversaries",
    "2 agents avoid 4 adversaries",
    "6 agents chase 3 adversaries",
    "3 agents avoid 6 adversaries",
    # "5 agents push 5 enemies away from 5 landmarks",
    "Agents employ various strategies and tactics to effectively deter adversaries from approaching or attacking significant landmarks.",
]
codes = [
    "011100001000",
    "100100001000",
    "101100001000",
    "011010000100",
    "100010000100",
    "101010000100",
    "011001001000",
    "100001001000",
    "101001001000",
    "100000101000",
    "010000011000",
    "110000101000",
    "011000011000",
    "101111111000",
]
message_embeddings=torch.rand(len(messages),512)
data = {}
for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print(message_embedding)
    data[codes[i]] = message_embedding
    # data[messages[i]] = message_embedding

with open('random.json', 'w') as file:
  json.dump(data, file)


# Try llama


In [ ]:
!huggingface-cli login

In [ ]:
pip install -U "transformers==4.40.0" --upgrade


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


More detailed information about installing Tensorflow can be found at [https://www.tensorflow.org/install/](https://www.tensorflow.org/install/).

In [ ]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

In [ ]:
#@title Compute a representation for each message, showing various lengths supported.
# word = "Elephant"
# word = "大象"
# sentence = "I am a sentence for which I would like to get its embedding."
# paragraph = (
#     "Universal Sentence Encoder embeddings also support short paragraphs. "
#     "There is no hard limit on how long the paragraph is. Roughly, the longer "
#     "the more 'diluted' the embedding will be.")
# messages = [word, sentence, paragraph]
import json
messages = [
    "3 agents occupy 3 landmarks",
    "4 agents occupy 4 landmarks",
    "5 agents occupy 5 landmarks",
    "3 agents form a regular polygon near the landmark",
    "4 agents form a regular polygon near the landmark",
    "5 agents form a regular polygon near the landmark",
    "3 agents compete with 3 adversaries to occupy landmarks",
    "4 agents compete with 4 adversaries to occupy landmarks",
    "5 agents compete with 5 adversaries to occupy landmarks",
    "4 agents chase 2 adversaries",
    "2 agents avoid 4 adversaries",
    "6 agents chase 3 adversaries",
    "3 agents avoid 6 adversaries",
    # "5 agents push 5 enemies away from 5 landmarks",
    "Agents employ various strategies and tactics to effectively deter adversaries from approaching or attacking significant landmarks.",
]
codes = [
    "011100001000",
    "100100001000",
    "101100001000",
    "011010000100",
    "100010000100",
    "101010000100",
    "011001001000",
    "100001001000",
    "101001001000",
    "100000101000",
    "010000011000",
    "110000101000",
    "011000011000",
    "101111111000",
]
# messages = [
#     # "agents occupy landmarks",
#     # "agents form a regular polygon near the landmark",
#     # "agents compete with enemies to occupy landmarks",
#     # "agents chase enemies",
#     # "agents avoid enemies",
#     # "agents push enemies away from landmarks",
#     "four agents form a regular polygon near the landmark",
#     "five agents form a regular polygon near the landmark",
# ]

# Reduce logging output.
logging.set_verbosity(logging.ERROR)

message_embeddings = embed(messages)
data = {}
for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print(message_embedding)
    data[codes[i]] = message_embedding
    # data[messages[i]] = message_embedding

with open('code2use.json', 'w') as file:
  json.dump(data, file)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Semantic Textual Similarity Task Example

The embeddings produced by the Universal Sentence Encoder are approximately normalized. The semantic similarity of two sentences can be trivially computed as the inner product of the encodings.

## Similarity Visualized
Here we show the similarity in a heat map. The final graph is a 9x9 matrix where each entry `[i, j]` is colored based on the inner product of the encodings for sentence `i` and `j`.

In [ ]:
# def plot_similarity(labels, features, rotation):
#   corr = np.inner(features, features)
#   sns.set(font_scale=1.2)
#   g = sns.heatmap(
#       corr,
#       xticklabels=labels,
#       yticklabels=labels,
#       vmin=0,
#       vmax=1,
#       cmap="YlOrRd")
#   g.set_xticklabels(labels, rotation=rotation)
#   g.set_title("Semantic Textual Similarity")
#   plt.savefig("similarity.png", dpi=300)


def plot_similarity(labels, features, rotation, dpi=300):
  # labels = range(1, len(labels)+1)
  labels = ["Spread", "Form Shape", "Push", "Tag 1", "Tag 2", "Push Away"]
  corr = np.inner(features, features)
  sns.set(font_scale=1.2)
  # 绘制热力图
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      # cmap="YlOrRd",
      cmap="coolwarm")
  g.set_xticklabels(labels, rotation=45)
  # g.set_title("Semantic Textual Similarity")
  # # 将第12个标签设置为红色
  # for i, label in enumerate(g.get_xticklabels()):
  #   if i == 11:
  #     label.set_color('green')
  # for i, label in enumerate(g.get_yticklabels()):
  #   if i == 11:
  #     label.set_color('green')

  plt.savefig("similarity.png", dpi=dpi)
  plt.savefig("similarity.svg", dpi=dpi)


def run_and_plot(messages_):
  message_embeddings_ = embed(messages_)
  for i in range(len(message_embeddings_)):
    if i != 5: continue
    print(message_embeddings_[i])
    for j in range(len(message_embeddings_)):
      print(i, j, np.inner(message_embeddings_[i], message_embeddings_[j]))
  plot_similarity(messages_, message_embeddings_, 90)

In [ ]:
messages = [
    # "3 agents occupy 3 landmarks",
    # "4 agents occupy 4 landmarks",
    # "5 agents occupy 5 landmarks",
    # "3 agents form a regular polygon near the landmark",
    # "4 agents form a regular polygon near the landmark",
    # "5 agents form a regular polygon near the landmark",
    # "3 agents compete with 3 adversaries to occupy landmarks",
    # "4 agents compete with 4 adversaries to occupy landmarks",
    # "5 agents compete with 5 adversaries to occupy landmarks",
    # "4 agents chase 2 adversaries",
    # "2 agents avoid 4 adversaries",
    # # "6 agents chase 3 adversaries",
    # # "3 agents avoid 6 adversaries",
    # "5 agents push 5 adversaries away from 5 landmarks",
    "agents occupy landmarks",
    "agents form a regular polygon near the landmark",
    "agents compete with enemies to occupy landmarks",
    "agents chase enemies",
    "agents avoid enemies",
    # "agents keep enemies away from landmarks",
    "Agents employ various strategies and tactics to effectively deter adversaries from approaching or attacking significant landmarks.",
]

run_and_plot(messages)


## Evaluation: STS (Semantic Textual Similarity) Benchmark

The [**STS Benchmark**](https://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) provides an intrinsic evaluation of the degree to which similarity scores computed using sentence embeddings align with human judgements. The benchmark requires systems to return similarity scores for a diverse selection of sentence pairs. [Pearson correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) is then used to evaluate the quality of the machine similarity scores against human judgements.

### Download data

In [ ]:
import pandas
import scipy
import math
import csv

sts_dataset = tf.keras.utils.get_file(
    fname="Stsbenchmark.tar.gz",
    origin="http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz",
    extract=True)
sts_dev = pandas.read_table(
    os.path.join(os.path.dirname(sts_dataset), "stsbenchmark", "sts-dev.csv"),
    error_bad_lines=False,
    skip_blank_lines=True,
    usecols=[4, 5, 6],
    names=["sim", "sent_1", "sent_2"])
sts_test = pandas.read_table(
    os.path.join(
        os.path.dirname(sts_dataset), "stsbenchmark", "sts-test.csv"),
    error_bad_lines=False,
    quoting=csv.QUOTE_NONE,
    skip_blank_lines=True,
    usecols=[4, 5, 6],
    names=["sim", "sent_1", "sent_2"])
# cleanup some NaN values in sts_dev
sts_dev = sts_dev[[isinstance(s, str) for s in sts_dev['sent_2']]]

### Evaluate Sentence Embeddings

In [ ]:
sts_data = sts_dev #@param ["sts_dev", "sts_test"] {type:"raw"}

def run_sts_benchmark(batch):
  sts_encode1 = tf.nn.l2_normalize(embed(tf.constant(batch['sent_1'].tolist())), axis=1)
  sts_encode2 = tf.nn.l2_normalize(embed(tf.constant(batch['sent_2'].tolist())), axis=1)
  cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
  clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
  scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
  """Returns the similarity scores"""
  return scores

dev_scores = sts_data['sim'].tolist()
scores = []
for batch in np.array_split(sts_data, 10):
  scores.extend(run_sts_benchmark(batch))

pearson_correlation = scipy.stats.pearsonr(scores, dev_scores)
print('Pearson correlation coefficient = {0}\np-value = {1}'.format(
    pearson_correlation[0], pearson_correlation[1]))